# DB Initialisation Script

This notebook can be used for setting up local Postgres database called 'amazon' and for querying the database

To get started, please change the env variables in the next code chunk and simply 'Run All' to load the database.

Ensure that your .venv is activated in this Jupyter environment.

In [1]:
path_to_products_csv = 'products.csv'
path_to_online_sales_csv = 'online_sales_edited.csv'

## Set Up

In [2]:
from sqlalchemy import create_engine, Table, Column, MetaData, String, Numeric, Float, Integer, ForeignKey, Date, ForeignKeyConstraint,DateTime
import pandas as pd
import psycopg2
from dotenv import load_dotenv
import os

In [3]:
# Load from .env file
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

load_dotenv(f'{parent_dir}/.env')

postgres_password = os.getenv('POSTGRES_PASSWORD')
postgres_port_no = os.getenv('POSTGRES_PORT')
host = os.getenv('POSTGRES_HOST')
database = os.getenv('POSTGRES_DB')
user = os.getenv('POSTGRES_USER')

In [4]:
# Step 1: Connect to the default database (postgres) to check/create the 'amazon' database
try:
    # Connect to the 'postgres' database (default)
    conn = psycopg2.connect(
        host=host,
        user=user,
        password=postgres_password,
        port=postgres_port_no,
        database='postgres'  # Connect to 'postgres' instead of 'amazon' initially
    )
    conn.autocommit = True  # Enable autocommit so that CREATE DATABASE works
    cursor = conn.cursor()

    # Check if the 'amazon' database exists
    cursor.execute(f"SELECT 1 FROM pg_database WHERE datname = '{database}';")
    exists = cursor.fetchone()

    # If the database doesn't exist, create it
    if not exists:
        cursor.execute(f'CREATE DATABASE {database};')
        print(f"Database '{database}' has been created.")
    else:
        print(f"Database '{database}' already exists.")
    
    cursor.close()
    conn.close()

except Exception as e:
    print(f"Error while creating or checking the database: {e}")

# Step 2: Now connect to the newly created or existing 'amazon' database
try:
    engine = create_engine(f'postgresql://{user}:{postgres_password}@{host}:{postgres_port_no}/{database}')

    # Define MetaData for the DB
    metadata = MetaData()

    # Reflect existing tables from the 'amazon' database
    metadata.reflect(bind=engine)
    
    # Drop all tables in the 'amazon' database
    metadata.drop_all(bind=engine)
    print("All pre-existing tables have been dropped.")
    
except Exception as e:
    print(f"Error while reflecting or dropping tables: {e}")

finally:
    metadata = MetaData()


Database 'amazon' already exists.
All pre-existing tables have been dropped.


## Loading tables into database

In [5]:
# Define the 'products' table schema
products_table = Table('products', metadata,
    Column('product_id', String(50), primary_key=True),
    Column('product_name', String(500)),
    Column('about_product', String),  # Use String for TEXT in SQLAlchemy
    Column('category', String(255)),
    Column('actual_price', Numeric(10, 2)),
    Column('discounted_price', Numeric(10, 2)),
    Column('discount_percentage', Numeric(5, 2)),
    Column('origin_area', String(50)),
    Column('img_link', String(500))
)

# Define the 'online_sales' table schema
online_sales_table = Table('online_sales', metadata,
    Column('cust_id', Integer, primary_key=True),
    Column('transaction_id', Integer, primary_key=True),
    Column('date', Date),
    Column('product_id', String(50), primary_key=True),
    Column('coupon_status', String(50), primary_key=True),
    Column('coupon_code', String(50), primary_key=True),
    Column('discount_percentage', Numeric(5,2)),  
    Column('delivery_charges', Numeric(10, 2)),
    Column('quantity', Integer),
    ForeignKeyConstraint(['product_id'],['products.product_id'],ondelete='CASCADE')
)

# Create the table in the database if it doesn't exist
metadata.create_all(engine)

In [6]:
# Reading the csvs
products_df = pd.read_csv(path_to_products_csv)
online_sales_df = pd.read_csv(path_to_online_sales_csv)

# Convert 'date' column to correct format if necessary
online_sales_df['date'] = pd.to_datetime(online_sales_df['date'], format='%Y-%m-%d', errors='raise').dt.date

In [7]:
# Insert data into the PostgreSQL table
products_df.to_sql('products', engine, if_exists='append', index=False)
online_sales_df.to_sql('online_sales', engine, if_exists='append', index=False)

457